<h2>Conflict and Disaster API Usage and Data Scraping</h2>
<h4>Using the <a href="https://apidocs.acleddata.com/get_started.html">ACLED API</a></h4>
<p>This data was originally accessed on March 1, 2024 and will be continually updated as part of a data pipeline in the backend of a model-mapping web application. Armed Conflict Location & Event Data Project (ACLED); www.acleddata.com</p>

<a href="https://acleddata.com/acleddatanew/wp-content/uploads/dlm_uploads/2023/06/ACLED_Codebook_2023.pdf">Data Columns Codebook</a>

In [35]:
# General imports
import pandas as pd
import numpy as np
import asyncio
import aiohttp
import requests
import json
from io import StringIO
from typing import Union

In [46]:
# Setting global API Variables
BASE_URL = "https://api.acleddata.com/"
ACLED_API_KEY = "VBNw-*4nvzHEC87xu2qQ"
USER_EMAIL = "nabji@berkeley.edu"
BASE_PARAMS = {
    "email": USER_EMAIL,
    "key": ACLED_API_KEY,
}

fields = [
    "event_id_cnty",
    "event_date",
    "year",
    "time_precision",
    "disorder_type",
    "event_type",
    "sub_event_type",
    "actor1",
    "assoc_actor_1",
    "inter1",
    "actor2",
    "assoc_actor_2",
    "inter2",
    "interaction",
    "civilian_targeting",
    "iso",
    "region",
    "country",
    "admin1",
    "admin2",
    "admin3",
    "location",
    "latitude",
    "longitude",
    "geo_precision",
    "source",
    "source_scale",
    "notes",
    "fatalities",
    "tags",
    "timestamp"
]

BASE_PARAMS["fields"] = "|".join(fields)
BASE_PARAMS

{'email': 'nabji@berkeley.edu',
 'key': 'VBNw-*4nvzHEC87xu2qQ',
 'fields': 'event_id_cnty|event_date|year|time_precision|disorder_type|event_type|sub_event_type|actor1|assoc_actor_1|inter1|actor2|assoc_actor_2|inter2|interaction|civilian_targeting|iso|region|country|admin1|admin2|admin3|location|latitude|longitude|geo_precision|source|source_scale|notes|fatalities|tags|timestamp'}

In [50]:
# Creating a function to request data from ACLED
def get_ACLED_data(country: str, year: int) -> Union[pd.DataFrame, bool]:
    """A function which, given a country and year will return all fields of ACLED satisfying those parameters
    
    Parameters
    ----------
    country: str
        The country from which we are requesting data
    year: int
        The year from which we are requesting data. year <= 2024
    
    Return Value
    ------------
    type: pd.DataFrame | bool
        A Pandas DataFrame containing information for all ACLED columns satisying the given parameters. If the request fails, return False
    """
    # Modify the base parameters
    new_params = BASE_PARAMS.copy()
    new_params["country"] = country
    new_params["year"] = year
    
    # Request the data as a CSV file and pass our paramenters as an argument (params=)
    response = requests.get("https://api.acleddata.com/acled/read.csv?", params=new_params)
    if response.status_code == 200:
        csv_data = StringIO(response.text)
        return_df = pd.read_csv(csv_data)
        return return_df
    else:
        return False

In [51]:
get_ACLED_data(country="Georgia", year=2023)

,event_id_cnty,event_date,year,time_precision,disorder_type,event_type,sub_event_type,actor1,assoc_actor_1,inter1,...,location,latitude,longitude,geo_precision,source,source_scale,notes,fatalities,tags,timestamp
0,GEO4295,2023-12-27,2023,1,Demonstrations,Protests,Peaceful protest,Protesters (Georgia),NaN,6,...,Batumi,41.6423,41.6339,1,Caucasian Knot,International,"On 27 December 2023, protesters demanding hous...",0,crowd size=no report,1704833925
1,GEO4294,2023-12-26,2023,1,Demonstrations,Protests,Peaceful protest,Protesters (Georgia),NaN,6,...,Sukhumi,43.0020,41.0139,1,Caucasian Knot,International,"On 26 December 2023, a protest was organized i...",0,crowd size=no report,1704833924
2,GEO4293,2023-12-23,2023,1,Demonstrations,Protests,Peaceful protest,Protesters (Georgia),NaN,6,...,Tbilisi,41.7183,44.8306,1,Black Sea Press: News Georgia,National,"On 23 December 2023, activists demanded free f...",0,crowd size=no report,1704833924
3,GEO4292,2023-12-20,2023,1,Demonstrations,Protests,Peaceful protest,Protesters (Georgia),ENM: United National Movement,6,...,Tbilisi,41.7183,44.8306,1,Interpressnews,National,"On 20 December 2023, the United National Movem...",0,crowd size=no report,1704833924
4,GEO4291,2023-12-15,2023,1,Demonstrations,Protests,Peaceful protest,Protesters (Georgia),Government of Georgia (2012-); Georgian Dream,6,...,Tbilisi,41.7183,44.8306,1,Caucasian Knot,International,"On 15 December 2023, thousands marched in Tbil...",0,crowd size=thousands,1704833922
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313,GEO3980,2023-01-04,2023,1,Demonstrations,Protests,Peaceful protest,Protesters (Georgia),ENM: United National Movement,6,...,Shekvetili,41.9305,41.7686,1,Interpressnews,National,"On 4 January 2023, the United National Movemen...",0,crowd size=no report,1673376401
314,GEO3981,2023-01-04,2023,1,Demonstrations,Protests,Peaceful protest,Protesters (Georgia),ENM: United National Movement,6,...,Tbilisi,41.7183,44.8306,1,Agenda.ge,National,"On 4 January 2023, the United National Movemen...",0,crowd size=no report,1673376401
315,GEO3982,2023-01-04,2023,1,Demonstrations,Protests,Peaceful protest,Protesters (Georgia),NaN,6,...,Zugdidi,42.5088,41.8709,1,RFE/RL,International,"On 4 January 2023, supporters of Mikheil Saaka...",0,crowd size=no report,1673376401
316,GEO3983,2023-01-04,2023,1,Demonstrations,Riots,Violent demonstration,Rioters (Georgia),ENM: United National Movement,5,...,Chorvila,42.2830,43.4127,1,Caucasian Knot,International,"On 4 January 2023, the United National Movemen...",0,crowd size=no report,1702344162
